In [1]:
#this notebook is mean to analyze pairs of assets ,their correlation and cointegration
import sys
sys.path.append(r"e:\Coding Projects\Investment Analysis")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
import yfinance as yf
from plotly.subplots import make_subplots
import plotly.graph_objects as go


#SP500
asset_1 = 'XLV' 

asset_2 = 'SPY'

period = '10y'
interval = '1d'


asset_1_data = yf.Ticker(asset_1).history(period=period, interval=interval)
asset_2_data = yf.Ticker(asset_2).history(period=period, interval=interval)

asset_1_close = asset_1_data['Close']
asset_2_close = asset_2_data['Close']

asset_1_close_log_prices = np.log(asset_1_close)
asset_2_close_log_prices = np.log(asset_2_close)

asset_1_close_returns_50 = asset_1_close.pct_change(50).dropna()
asset_2_close_returns_50 = asset_2_close.pct_change(50).dropna()

asset_1_close_returns_200 = asset_1_close.pct_change(200).dropna()
asset_2_close_returns_200 = asset_2_close.pct_change(200).dropna()

# Create a figure with 4 subplots - 1 for log prices, 2 for returns (50d and 200d), 2 for spread z-scores
fig = make_subplots(rows=3, cols=2, 
                   subplot_titles=['Log Prices', '50-Day Returns', '200-Day Returns', f'{asset_1} - {asset_2} (50-Day Z-Score)', f'{asset_1} - {asset_2} (200-Day Z-Score)'],
                   specs=[[{"colspan": 2}, None],
                          [{"colspan": 1}, {"colspan": 1}],
                          [{"colspan": 1}, {"colspan": 1}]],
                   vertical_spacing=0.1,
                   horizontal_spacing=0.05)

# Add the log prices traces to the first subplot (spanning both columns)
fig.add_trace(
    go.Scatter(x=asset_1_close_log_prices.index, y=asset_1_close_log_prices, name=asset_1, line=dict(color='red')),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=asset_2_close_log_prices.index, y=asset_2_close_log_prices, name=asset_2, line=dict(color='blue')),
    row=1, col=1
)

# Add the 50-day returns traces to the left subplot in the second row
fig.add_trace(
    go.Scatter(x=asset_1_close_returns_50.index, y=asset_1_close_returns_50, name=f"{asset_1} (50d)", line=dict(color='red')),
    row=2, col=1
)
fig.add_trace(
    go.Scatter(x=asset_2_close_returns_50.index, y=asset_2_close_returns_50, name=f"{asset_2} (50d)", line=dict(color='blue')),
    row=2, col=1
)

# Add the 200-day returns traces to the right subplot in the second row
fig.add_trace(
    go.Scatter(x=asset_1_close_returns_200.index, y=asset_1_close_returns_200, name=f"{asset_1} (200d)", line=dict(color='red')),
    row=2, col=2
)
fig.add_trace(
    go.Scatter(x=asset_2_close_returns_200.index, y=asset_2_close_returns_200, name=f"{asset_2} (200d)", line=dict(color='blue')),
    row=2, col=2
)

# Calculate spreads
spread_50d = asset_1_close_returns_50 - asset_2_close_returns_50
spread_200d = asset_1_close_returns_200 - asset_2_close_returns_200

# Calculate z-scores
spread_50d_mean = spread_50d.mean()
spread_50d_std = spread_50d.std()
z_score_50d = (spread_50d - spread_50d_mean) / spread_50d_std

spread_200d_mean = spread_200d.mean()
spread_200d_std = spread_200d.std()
z_score_200d = (spread_200d - spread_200d_mean) / spread_200d_std

# Add the 50-day z-score trace to the left subplot in the third row
fig.add_trace(
    go.Scatter(x=z_score_50d.index, y=z_score_50d, 
               name='Z-Score (50d)', line=dict(color='blue')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=z_score_50d.index, y=[0]*len(z_score_50d), 
               name='Zero', line=dict(color='black', dash='dash')),
    row=3, col=1
)
# Add standard deviation lines for 50-day z-score
for i, sd in enumerate([1, 2, 3]):
    fig.add_trace(
        go.Scatter(x=z_score_50d.index, y=[sd]*len(z_score_50d), 
                   name=f'+{sd} SD', line=dict(color='green', dash='dot', width=1)),
        row=3, col=1
    )
    fig.add_trace(
        go.Scatter(x=z_score_50d.index, y=[-sd]*len(z_score_50d), 
                   name=f'-{sd} SD', line=dict(color='green', dash='dot', width=1)),
        row=3, col=1
    )

# Add annotations for trading signals on 50-day z-score
fig.add_annotation(
    x=z_score_50d.index[len(z_score_50d)//2],
    y=2.2,
    text=f"Short {asset_1}, Long {asset_2}",
    showarrow=False,
    font=dict(color="red", size=12),
    bgcolor="rgba(255,255,255,0.8)",
    row=3, col=1
)
fig.add_annotation(
    x=z_score_50d.index[len(z_score_50d)//2],
    y=-2.2,
    text=f"Long {asset_1}, Short {asset_2}",
    showarrow=False,
    font=dict(color="green", size=12),
    bgcolor="rgba(255,255,255,0.8)",
    row=3, col=1
)

# Add the 200-day z-score trace to the right subplot in the third row
fig.add_trace(
    go.Scatter(x=z_score_200d.index, y=z_score_200d, 
               name='Z-Score (200d)', line=dict(color='red')),
    row=3, col=2
)
fig.add_trace(
    go.Scatter(x=z_score_200d.index, y=[0]*len(z_score_200d), 
               name='Zero', line=dict(color='black', dash='dash')),
    row=3, col=2
)

# Add standard deviation lines for 200-day z-score
for i, sd in enumerate([1, 2, 3]):
    fig.add_trace(
        go.Scatter(x=z_score_200d.index, y=[sd]*len(z_score_200d), 
                   name=f'+{sd} SD (200d)', line=dict(color='orange', dash='dot', width=1)),
        row=3, col=2
    )
    fig.add_trace(
        go.Scatter(x=z_score_200d.index, y=[-sd]*len(z_score_200d), 
                   name=f'-{sd} SD (200d)', line=dict(color='orange', dash='dot', width=1)),
        row=3, col=2
    )

# Add annotations for trading signals on 200-day z-score
fig.add_annotation(
    x=z_score_200d.index[len(z_score_200d)//2],
    y=2.2,
    text=f"Short {asset_1}, Long {asset_2}",
    showarrow=False,
    font=dict(color="red", size=12),
    bgcolor="rgba(255,255,255,0.8)",
    row=3, col=2
)
fig.add_annotation(
    x=z_score_200d.index[len(z_score_200d)//2],
    y=-2.2,
    text=f"Long {asset_1}, Short {asset_2}",
    showarrow=False,
    font=dict(color="green", size=12),
    bgcolor="rgba(255,255,255,0.8)",
    row=3, col=2
)

# Update the layout with proper axis titles
fig.update_xaxes(title_text="Date", row=1, col=1)
fig.update_xaxes(title_text="Date", row=2, col=1)
fig.update_xaxes(title_text="Date", row=2, col=2)
fig.update_xaxes(title_text="Date", row=3, col=1)
fig.update_xaxes(title_text="Date", row=3, col=2)

fig.update_yaxes(title_text="Log Price", row=1, col=1)
fig.update_yaxes(title_text="Returns", row=2, col=1)
fig.update_yaxes(title_text="Returns", row=2, col=2)
fig.update_yaxes(title_text="Z-Score", row=3, col=1)
fig.update_yaxes(title_text="Z-Score", row=3, col=2)

# Update the layout
fig.update_layout(
    height=1500, 
    title_text=f"{asset_1} vs {asset_2} Analysis",
    showlegend=True,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)
fig.show()

